In [1]:
import torch
import numpy as np
from importlib import reload
import librosa

In [2]:
speech_path = 'assets/Folly Audio/Meditation VO - Mindful Beathing.mp3'
music_path = 'assets/Folly Audio/Meditation Music_moderatearousal.mp3'

In [3]:
# import src.speech
# reload(src.speech)
# from src.speech import SpeechInformationRetreiver


# sir = SpeechInformationRetreiver(
#     model_name='stt_en_fastconformer_hybrid_large_streaming_1040ms',
#     lookahead_size=1040,
#     decoder_type='rnnt',
#     num_keywords=10,
#     device='cuda'
# )
# speech_segments = sir(speech_path, max_dist=5., extract_kw=True, verbose=True)

In [4]:
# from src.music.mir import MusicInformationRetreiver
# from src.music.bridge import MusicBridge


# mir = MusicInformationRetreiver(weights_path='mir_weights/', segment_threshold=0.1)
# music_segments = mir(music_path, class_threshold=0.3, verbpose=True)

In [5]:
music_segments = [{'start': 0,
  'end': 139.84,
  'duration': 139.84,
  'valence': 0.0005982993,
  'arousal': 0.001197161,
  'text': 'ambient, electronic, piano, synthesizer'},
 {'start': 139.84,
  'end': 291.072,
  'duration': 151.232,
  'valence': 0.0008024101,
  'arousal': 0.0016852224,
  'text': 'ambient, chillout, electronic, piano, synthesizer'},
 {'start': 291.072,
  'end': 426.08,
  'duration': 135.00799999999998,
  'valence': 0.00031344328,
  'arousal': 0.0008951402,
  'text': 'ambient, chillout, electronic, synthesizer'},
 {'start': 426.08,
  'end': 560.736,
  'duration': 134.656,
  'valence': 0.0019070618,
  'arousal': 0.0024938928,
  'text': 'ambient, chillout, electronic, piano, synthesizer'},
 {'start': 560.736,
  'end': 632.16,
  'duration': 71.42399999999998,
  'valence': 0.0007020948,
  'arousal': 0.0014097905,
  'text': 'ambient, chillout, electronic, piano, synthesizer'},
 {'start': 632.16,
  'end': 873.44,
  'duration': 241.2800000000001,
  'valence': 0.002704294,
  'arousal': 0.0037417596,
  'text': 'ambient, bass, piano'},
 {'start': 873.44,
  'end': 936.059875,
  'duration': 62.61987499999998,
  'valence': 0.0033889317,
  'arousal': 0.0044924943,
  'text': 'piano'}]

In [6]:
speech_segments = [{'text': 'hello and welcome to your resonance r xes',
  'start': 35.85590062111801,
  'end': 41.4583850931677,
  'duration': 5.602484472049689},
 {'text': ' guided meditation',
  'start': 42.57888198757764,
  'end': 43.69937888198758,
  'duration': 1.1204968944099392},
 {'text': ' hi mandrew',
  'start': 47.06086956521739,
  'end': 48.18136645962733,
  'duration': 1.1204968944099392},
 {'text': ' i am here to guide you through a transformative journey',
  'start': 50.422360248447205,
  'end': 52.66335403726708,
  'duration': 2.240993788819871},
 {'text': ' where we will be embracing',
  'start': 54.904347826086955,
  'end': 56.024844720496894,
  'duration': 1.1204968944099392},
 {'text': ' the powerful practice of mindfulness',
  'start': 57.145341614906826,
  'end': 61.62732919254658,
  'duration': 4.481987577639757},
 {'text': ' focusing on our breath',
  'start': 62.74782608695652,
  'end': 67.22981366459626,
  'duration': 4.481987577639742},
 {'text': ' this practice',
  'start': 73.9527950310559,
  'end': 75.07329192546584,
  'duration': 1.1204968944099392},
 {'text': ' is designed with you and mine',
  'start': 77.31428571428572,
  'end': 78.43478260869564,
  'duration': 1.120496894409925},
 {'text': ' aiming to bring moments',
  'start': 81.79627329192546,
  'end': 82.9167701863354,
  'duration': 1.1204968944099392},
 {'text': ' of peace',
  'start': 84.03726708074534,
  'end': 85.15776397515528,
  'duration': 1.1204968944099392},
 {'text': ' relaxation',
  'start': 87.39875776397515,
  'end': 88.5192546583851,
  'duration': 1.1204968944099392},
 {'text': ' and clarity',
  'start': 89.63975155279502,
  'end': 90.76024844720496,
  'duration': 1.1204968944099392},
 {'text': " let's start by finding a comfortable spot",
  'start': 99.72422360248447,
  'end': 103.08571428571427,
  'duration': 3.3614906832298033},
 {'text': ' where you can sit',
  'start': 104.20621118012421,
  'end': 105.32670807453415,
  'duration': 1.1204968944099392},
 {'text': ' or lie down',
  'start': 106.44720496894409,
  'end': 107.56770186335403,
  'duration': 1.1204968944099392},
 {'text': ' ensuring that distractions are minimal',
  'start': 108.68819875776397,
  'end': 112.04968944099379,
  'duration': 3.3614906832298175},
 {'text': " once you've settled that your arms rest naturally",
  'start': 117.65217391304347,
  'end': 122.13416149068323,
  'duration': 4.481987577639757},
 {'text': ' and have your legs in a relaxed position',
  'start': 123.25465838509317,
  'end': 126.61614906832297,
  'duration': 3.3614906832298033},
 {'text': ' has we embark on this journey',
  'start': 161.35155279503104,
  'end': 163.59254658385092,
  'duration': 2.2409937888198783},
 {'text': ' begin by taking a few profound and rejuvenating breaths',
  'start': 164.71304347826086,
  'end': 170.31552795031055,
  'duration': 5.602484472049696},
 {'text': ' inhal deeply',
  'start': 173.67701863354037,
  'end': 175.91801242236025,
  'duration': 2.2409937888198783},
 {'text': ' feeling your chest',
  'start': 177.0385093167702,
  'end': 178.15900621118013,
  'duration': 1.1204968944099392},
 {'text': ' and abdomen rise',
  'start': 179.27950310559004,
  'end': 180.39999999999998,
  'duration': 1.1204968944099392},
 {'text': ' and then exhale slowly',
  'start': 183.7614906832298,
  'end': 186.00248447204967,
  'duration': 2.2409937888198783},
 {'text': ' letting go if any strains',
  'start': 187.1229813664596,
  'end': 189.3639751552795,
  'duration': 2.2409937888198783},
 {'text': ' or worries with every breath',
  'start': 190.48447204968943,
  'end': 192.7254658385093,
  'duration': 2.2409937888198783},
 {'text': ' do this a couple times',
  'start': 197.20745341614906,
  'end': 199.44844720496894,
  'duration': 2.2409937888198783},
 {'text': ' immersing yourself in this present moment',
  'start': 200.56894409937888,
  'end': 203.93043478260867,
  'duration': 3.361490683229789},
 {'text': ' disconnecting from any external stresses',
  'start': 208.41242236024843,
  'end': 211.77391304347825,
  'duration': 3.3614906832298175},
 {'text': ' feel the connection between your body',
  'start': 215.13540372670806,
  'end': 217.37639751552794,
  'duration': 2.2409937888198783},
 {'text': ' and its resting place',
  'start': 218.49689440993788,
  'end': 220.73788819875776,
  'duration': 2.2409937888198783},
 {'text': ' and the grounding sensation',
  'start': 222.97888198757764,
  'end': 225.21987577639752,
  'duration': 2.2409937888198783},
 {'text': ' of your feet touching the floor',
  'start': 226.34037267080745,
  'end': 229.70186335403724,
  'duration': 3.361490683229789},
 {'text': " our breath will be the anchor of today's meditation",
  'start': 292.4496894409938,
  'end': 298.0521739130435,
  'duration': 5.602484472049696},
 {'text': ' without altering it',
  'start': 300.29316770186335,
  'end': 302.53416149068323,
  'duration': 2.2409937888198783},
 {'text': ' observe its current rhythm',
  'start': 303.65465838509317,
  'end': 305.89565217391305,
  'duration': 2.2409937888198783},
 {'text': ' is it shallow',
  'start': 308.1366459627329,
  'end': 309.25714285714287,
  'duration': 1.1204968944099392},
 {'text': ' deep',
  'start': 311.4981366459627,
  'end': 312.6186335403726,
  'duration': 1.1204968944099392},
 {'text': " fill the breath's journey as it enters and exits",
  'start': 317.1006211180124,
  'end': 321.58260869565214,
  'duration': 4.481987577639757},
 {'text': ' creating a rhythmic dance',
  'start': 323.823602484472,
  'end': 324.94409937888196,
  'duration': 1.1204968944099392},
 {'text': ' of inn',
  'start': 326.0645962732919,
  'end': 327.18509316770184,
  'duration': 1.1204968944099392},
 {'text': ' and about',
  'start': 328.3055900621118,
  'end': 329.4260869565217,
  'duration': 1.1204968944099392},
 {'text': ' reminiscent of gentle ocean waves',
  'start': 332.78757763975153,
  'end': 336.14906832298135,
  'duration': 3.3614906832298175},
 {'text': ' caressing the shore',
  'start': 337.2695652173913,
  'end': 339.51055900621117,
  'duration': 2.2409937888198783},
 {'text': ' present with this dance of breath',
  'start': 343.9925465838509,
  'end': 346.2335403726708,
  'duration': 2.2409937888198783},
 {'text': ' without any need to alter origin',
  'start': 347.35403726708074,
  'end': 350.71552795031056,
  'duration': 3.3614906832298175},
 {'text': ' simply',
  'start': 354.0770186335404,
  'end': 355.1975155279503,
  'duration': 1.1204968944099392},
 {'text': ' b',
  'start': 356.31801242236025,
  'end': 357.4385093167702,
  'duration': 1.1204968944099392},
 {'text': ' it is only human for our minds to wander',
  'start': 414.583850931677,
  'end': 417.9453416149068,
  'duration': 3.3614906832298175},
 {'text': ' and if it does',
  'start': 419.06583850931673,
  'end': 420.1863354037267,
  'duration': 1.1204968944099392},
 {'text': ' there is no need for concern',
  'start': 421.3068322981366,
  'end': 422.42732919254655,
  'duration': 1.1204968944099392},
 {'text': ' when you find yourself for mister whirlwind of thoughts or emotions',
  'start': 426.9093167701863,
  'end': 431.39130434782606,
  'duration': 4.481987577639757},
 {'text': ' acknowledge them gently',
  'start': 433.63229813664594,
  'end': 435.8732919254658,
  'duration': 2.2409937888198783},
 {'text': ' and redirect your focus to the comforting rhythm of your breast',
  'start': 436.99378881987576,
  'end': 443.7167701863354,
  'duration': 6.722981366459635},
 {'text': ' this practice is all about patience',
  'start': 448.19875776397515,
  'end': 451.56024844720497,
  'duration': 3.3614906832298175},
 {'text': ' kindness towards oneself',
  'start': 452.6807453416149,
  'end': 454.92173913043473,
  'duration': 2.2409937888198215},
 {'text': ' even if distractions emerge repeatedly',
  'start': 457.1627329192546,
  'end': 460.5242236024844,
  'duration': 3.3614906832298175},
 {'text': " that's perfectly fine",
  'start': 461.64472049689437,
  'end': 463.88571428571424,
  'duration': 2.2409937888198783},
 {'text': ' all that matters is the gentle return to the rhythm of your breath',
  'start': 467.24720496894406,
  'end': 472.84968944099376,
  'duration': 5.602484472049696},
 {'text': ' visualizes each breath',
  'start': 476.2111801242236,
  'end': 478.45217391304345,
  'duration': 2.2409937888198783},
 {'text': ' in',
  'start': 480.69316770186333,
  'end': 481.81366459627327,
  'duration': 1.1204968944099392},
 {'text': ' as a gentle reminder',
  'start': 485.1751552795031,
  'end': 487.41614906832297,
  'duration': 2.2409937888198783},
 {'text': ' of your commitment to this present moment',
  'start': 488.5366459627329,
  'end': 491.8981366459627,
  'duration': 3.3614906832298175},
 {'text': " let's now shift our attention to the area around your navel",
  'start': 534.4770186335403,
  'end': 538.95900621118,
  'duration': 4.481987577639757},
 {'text': ' feel the sensation of your abdomen moving with every inhalation',
  'start': 542.3204968944099,
  'end': 549.0434782608695,
  'duration': 6.722981366459635},
 {'text': ' descending with every exhalation',
  'start': 552.4049689440993,
  'end': 554.6459627329192,
  'duration': 2.2409937888198783},
 {'text': ' engaged deeply with these sensations',
  'start': 559.127950310559,
  'end': 561.3689440993788,
  'duration': 2.2409937888198783},
 {'text': ' perhaps mentally noting in on the inhale',
  'start': 562.4894409937888,
  'end': 566.9714285714285,
  'duration': 4.481987577639757},
 {'text': ' and',
  'start': 568.0919254658385,
  'end': 569.2124223602484,
  'duration': 1.1204968944099392},
 {'text': ' exhale',
  'start': 572.5739130434782,
  'end': 573.6944099378882,
  'duration': 1.1204968944099392},
 {'text': ' fortify your attention',
  'start': 574.8149068322981,
  'end': 575.935403726708,
  'duration': 1.1204968944099392},
 {'text': ' throughout this session',
  'start': 609.5503105590062,
  'end': 611.7913043478261,
  'duration': 2.2409937888198783},
 {'text': ' embrace everything surfaces with warmth and acceptance',
  'start': 612.911801242236,
  'end': 618.5142857142857,
  'duration': 5.602484472049696},
 {'text': ' of fleeting thought',
  'start': 621.8757763975154,
  'end': 622.9962732919254,
  'duration': 1.1204968944099392},
 {'text': ' an emotion',
  'start': 624.1167701863353,
  'end': 625.2372670807453,
  'duration': 1.1204968944099392},
 {'text': ' or a physical sensation',
  'start': 626.3577639751552,
  'end': 628.5987577639751,
  'duration': 2.2409937888198783},
 {'text': ' let it fall',
  'start': 629.719254658385,
  'end': 630.839751552795,
  'duration': 1.1204968944099392},
 {'text': ' when you identify them',
  'start': 634.2012422360248,
  'end': 636.4422360248446,
  'duration': 2.2409937888198783},
 {'text': ' rather than dwelling or resisting',
  'start': 637.5627329192546,
  'end': 639.8037267080745,
  'duration': 2.2409937888198783},
 {'text': ' kindly your focus back to your breath',
  'start': 642.0447204968943,
  'end': 645.4062111801242,
  'duration': 3.3614906832298175},
 {'text': ' these moments',
  'start': 647.647204968944,
  'end': 648.767701863354,
  'duration': 1.1204968944099392},
 {'text': ' these sensations',
  'start': 651.0086956521739,
  'end': 652.1291925465838,
  'duration': 1.1204968944099392},
 {'text': ' are all parts of this beautiful journey of mindfulness',
  'start': 653.2496894409937,
  'end': 657.7316770186335,
  'duration': 4.481987577639757},
 {'text': ' with practice',
  'start': 658.8521739130434,
  'end': 659.9726708074534,
  'duration': 1.1204968944099392},
 {'text': ' this acceptance becomes second nature',
  'start': 661.0931677018633,
  'end': 664.4546583850931,
  'duration': 3.3614906832298175},
 {'text': ' illuminating idea',
  'start': 665.5751552795031,
  'end': 666.695652173913,
  'duration': 1.1204968944099392},
 {'text': ' within a new found peace',
  'start': 667.8161490683229,
  'end': 670.0571428571428,
  'duration': 2.2409937888198783},
 {'text': ' as draw this meditation to a close',
  'start': 736.1664596273291,
  'end': 739.5279503105589,
  'duration': 3.3614906832298175},
 {'text': ' gradually bringing your consciousness',
  'start': 741.7689440993788,
  'end': 744.0099378881987,
  'duration': 2.2409937888198783},
 {'text': ' back to the space around you',
  'start': 745.1304347826086,
  'end': 747.3714285714285,
  'duration': 2.2409937888198783},
 {'text': ' if your eyes were closed',
  'start': 750.7329192546583,
  'end': 752.9739130434782,
  'duration': 2.2409937888198783},
 {'text': ' allow them to flutter open gently',
  'start': 754.0944099378881,
  'end': 757.455900621118,
  'duration': 3.3614906832298175},
 {'text': ' reflect for a moment on how you feel',
  'start': 760.8173913043478,
  'end': 764.1788819875776,
  'duration': 3.3614906832298175},
 {'text': ' compared to when you started',
  'start': 766.4198757763975,
  'end': 768.6608695652174,
  'duration': 2.2409937888198783},
 {'text': ' do you sense a shift',
  'start': 770.9018633540372,
  'end': 773.1428571428571,
  'duration': 2.2409937888198783},
 {'text': 'fulness offers us a chance to realign',
  'start': 784.3478260869565,
  'end': 787.7093167701863,
  'duration': 3.3614906832298175},
 {'text': ' providing a haven of calm acceptance',
  'start': 788.8298136645963,
  'end': 791.0708074534161,
  'duration': 2.2409937888198783},
 {'text': ' in our bustling lives',
  'start': 792.1913043478261,
  'end': 794.432298136646,
  'duration': 2.2409937888198783},
 {'text': " it's not about passive resignation",
  'start': 797.7937888198758,
  'end': 800.0347826086957,
  'duration': 2.2409937888198783},
 {'text': ' but about granting us the clarity to act with intention',
  'start': 801.1552795031056,
  'end': 805.6372670807453,
  'duration': 4.481987577639757},
 {'text': ' acceptance can beacon',
  'start': 808.9987577639752,
  'end': 812.360248447205,
  'duration': 3.3614906832298175},
 {'text': ' guiding us away from the needless stress',
  'start': 814.6012422360247,
  'end': 817.9627329192546,
  'duration': 3.3614906832298175},
 {'text': ' ushering in our realm of peace',
  'start': 821.3242236024844,
  'end': 823.5652173913043,
  'duration': 2.2409937888198783},
 {'text': ' thank yourself for dedicating this time to personal growth and tranquility',
  'start': 829.167701863354,
  'end': 835.8906832298136,
  'duration': 6.722981366459635},
 {'text': ' this practice of',
  'start': 840.3726708074533,
  'end': 842.6136645962732,
  'duration': 2.2409937888198783},
 {'text': ' always here',
  'start': 844.8546583850931,
  'end': 845.975155279503,
  'duration': 1.1204968944099392},
 {'text': ' waiting for you',
  'start': 847.095652173913,
  'end': 848.2161490683229,
  'duration': 1.1204968944099392},
 {'text': ' ready to guide you back to a moment of serenity whenever you need',
  'start': 849.3366459627329,
  'end': 856.0596273291925,
  'duration': 6.722981366459635},
 {'text': ' until an excession take care',
  'start': 858.3006211180124,
  'end': 861.6621118012422,
  'duration': 3.3614906832298175},
 {'text': ' let the soothing rhythms of life carry you forward',
  'start': 863.9031055900621,
  'end': 868.3850931677018,
  'duration': 4.481987577639757}]

In [7]:
for i,s in enumerate(speech_segments):
    s['style'] = 'Digital Art'
    speech_segments[i] = s

In [8]:
import src.video.gen
reload(src.video.gen)
from src.video.gen import VideoGenerator



video_gen = VideoGenerator()

/home/soroush/.venv/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/soroush/.venv/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
_ = video_gen.generate(
    speech_segments,
    music_segments, 
    speech_path, 
    music_path, 
    percussive_reactivity=False,
    batch_size=32, 
    duration=60, 
    guidance_scale=7.,
    num_inference_steps=20,
    fps=10,
    save_path='test.mp4'
)

Audio activities captured
latent vectors computed
starting generation...


  0%|          | 0/19 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a differen

Successfully merged video_file and audio_file into test.mp4


frame=  608 fps=326 q=-1.0 Lsize=   17182kB time=00:01:00.70 bitrate=2318.9kbits/s speed=32.6x    
video:15847kB audio:1312kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.135646%
[aac @ 0x55abbdaff600] Qavg: 6233.718


In [ ]:
from diffusers import AutoPipelineForInpainting

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", 
    torch_dtype=torch.float16, 
    variant="fp16",
    safety_checker=None
).to('cuda')
pipeline.enable_xformers_memory_efficient_attention()

/home/soroush/.venv/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/soroush/.venv/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/home/soroush/.venv/lib/python3.11/site-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoder

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pipeline.set_progress_bar_config(disable=True)

In [80]:
from PIL import Image
from tqdm.notebook import tqdm
from diffusers.utils import load_image, make_image_grid


def shrink_and_paste_on_blank(current_image, mask_width):
   
    height = current_image.height
    width = current_image.width

    # shrink down by mask_width
    prev_image = current_image.resize((height - 2 * mask_width, width - 2 * mask_width))
    prev_image = prev_image.convert("RGBA")
    prev_image = np.array(prev_image)

    # create blank non-transparent image
    blank_image = np.array(current_image.convert("RGBA")) * 0
    blank_image[:, :, 3] = 1

    # paste shrinked onto blank
    blank_image[mask_width : height - mask_width, mask_width : width - mask_width, :] = prev_image
    prev_image = Image.fromarray(blank_image)

    mask_image = np.array(prev_image)[:, :, 3]
    mask_image = Image.fromarray(255 - mask_image).convert("RGB")

    return prev_image.convert("RGB"), mask_image


duration = 60
fps = 10
n = duration * fps
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
frames = [init_image]
generator = torch.Generator("cuda").manual_seed(7)

for _ in tqdm(range(duration * fps)):
    curr, mask = shrink_and_paste_on_blank(frames[-1], 20)
    frames += pipeline(
        prompt='', 
        image=curr, 
        mask_image=mask, 
        generator=generator,
        # strength=0.999,
        num_inference_steps=20
    ).images



  0%|          | 0/600 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [79]:
import cv2

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # For MP4
video_writer = cv2.VideoWriter(
    'zoom.mp4', fourcc, fps, (512, 512)
)
for frame in frames:
    frame = np.array(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    video_writer.write(frame)
video_writer.release()